In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.chdir("/ultrasound-toolbox")

from usbmd.backend.tensorflow.dataloader import h5_dataset_from_directory
from usbmd import set_data_paths
import time

def _log(execution_time, batches):
    print("Execution time:", execution_time)
    print("Number of batches:", batches)
    print("Execution time per batch:", execution_time / batches)

def benchmark(dataset, num_epochs=2):
    batches = 0
    for epoch_num in range(num_epochs):
        if epoch_num == 0:
            start_epoch0 = time.perf_counter()
        if epoch_num == 1:
            start_time = time.perf_counter()
        for sample in iter(dataset):
            # Performing a training step
            batches += 1
            time.sleep(0.01)
        if epoch_num == 0:
            end_epoch0 = time.perf_counter()
            execution_time_epoch0 = end_epoch0 - start_epoch0
            print("------ Epoch == 0 ------")
            _log(execution_time_epoch0, batches)
            batches = 0
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print("------ Epoch >= 1 ------")
    _log(execution_time, batches)


def benchmark2(dataset, *args, **kwargs):
    times = [time.perf_counter()]
    images = []
    for sample in iter(dataset):
        time.sleep(0.01)
        times.append(time.perf_counter())
        images.append(sample)
        print(times[-1] - times[-2])
    return times, images


In [ ]:
# Set up data paths and device
data_paths = set_data_paths(local=False)
data_root = data_paths["data_root"]

dataset = h5_dataset_from_directory(
    data_root / "USBMD_datasets/echonet/val",
    key="data/image",
    batch_size=512,
    shuffle=True,
    resize_type="resize",
    seed=42,
    prefetch=False,
    cache=False,
    wrap_in_keras=True,
    limit_n_samples=10240, # 10 batches
)

times, images = benchmark2(dataset, num_epochs=2)

In [ ]:
import keras

keras.src.trainer.data_adapters.py_dataset_adapter.PyDatasetAdapter